In [20]:
import pandas as pd 
import numpy as np
import json
import os
import re
import shutil

In [2]:
pwd

'C:\\Users\\User\\LearnBay\\ML Projects\\Wafer Fault Detection'

In [6]:
ls

 Volume in drive C has no label.
 Volume Serial Number is B21D-C88B

 Directory of C:\Users\User\LearnBay\ML Projects\Wafer Fault Detection

03/25/2022  05:08 PM    <DIR>          .
03/25/2022  05:08 PM    <DIR>          ..
03/25/2022  05:02 PM    <DIR>          .ipynb_checkpoints
03/24/2022  06:27 PM    <DIR>          aaaaaaaaaaaa
03/24/2022  05:07 PM    <DIR>          Afsaan codes
03/24/2022  05:15 PM    <DIR>          Bad_data
03/24/2022  05:03 PM    <DIR>          good_data
03/24/2022  10:29 PM             9,756 part_1_data_injestion_validation.ipynb
03/23/2022  08:13 PM            17,188 schema_prediction.json
03/23/2022  08:13 PM            17,213 schema_training.json
03/23/2022  08:26 PM    <DIR>          Training_Batch_Files
03/25/2022  05:08 PM             2,461 Untitled.ipynb
               4 File(s)         46,618 bytes
               8 Dir(s)  139,803,762,688 bytes free


In [4]:
mkdir good_data

A subdirectory or file good_data already exists.


In [5]:
mkdir Bad_data

A subdirectory or file Bad_data already exists.


In [23]:
#Global Path
schema_file_path = 'C:\\Users\\User\\LearnBay\\ML Projects\\Wafer Fault Detection\\schema_training.json/'
training_folder_path = 'C:\\Users\\User\\LearnBay\\ML Projects\\Wafer Fault Detection\\Training_Batch_Files/'
good_data_folder_path = 'C:\\Users\\User\\LearnBay\\ML Projects\\Wafer Fault Detection\\good_data/'
bad_data_folder_path = 'C:\\Users\\User\\LearnBay\\ML Projects\\Wafer Fault Detection\\Bad_data/'

In [8]:
# extracting values from predicted schema

def valuesfromschema():
    with open(schema_file_path, 'r') as f:
        dic = json.load(f)
        pattern = dic['SampleFileName']
        LengthOfDateStampInFile = dic['LengthOfDateStampInFile']
        LengthOfTimeStampInFile = dic['LengthOfTimeStampInFile']
        column_names = dic['ColName']
        NumberofColumns = dic['NumberofColumns']
        
        return pattern, LengthOfDateStampInFile, LengthOfTimeStampInFile, column_names, NumberofColumns

In [9]:
pattern, LengthOfDateStampInFile, LengthOfTimeStampInFile, column_names, NumberofColumns = valuesfromschema()


# Regex

In [10]:
def regex():
    regex = "['wafer']+['\_'']+[\d_]+[\d]+\.csv"
    return regex

In [11]:
regex = regex()

In [24]:
# code for filtering the good and bad  files
def validate_file_name(regex, LengthOfDateStampInFile, LengthOfTimeStampInFile):
    for filename in os.listdir(training_folder_path):
        if (re.match(regex, filename)):
            split_at_dot = re.split('.csv', filename)
            split_at_dot = re.split('_', split_at_dot[0])
            if len(split_at_dot[1]) == LengthOfDateStampInFile:
                if len(split_at_dot[2]) == LengthOfTimeStampInFile:
                    shutil.copy(f'{training_folder_path}'+filename , good_data_folder_path)
                else:
                    shutil.copy(f'{training_folder_path}'+filename , bad_data_folder_path)
            else:
                shutil.copy(f'{training_folder_path}'+filename , bad_data_folder_path)
        else:
                shutil.copy(f'{training_folder_path}'+filename , bad_data_folder_path)
            
            
                


In [25]:
validate_file_name(regex, LengthOfDateStampInFile, LengthOfTimeStampInFile)

In [26]:
# validating column length in the file
def validatecolumn(NumberofColumns):
    for filename in os.listdir(good_data_folder_path):
        csv = pd.read_csv(f'{good_data_folder_path}'+filename)
        if csv.shape[1] == NumberofColumns:
            pass
        else:
            shutil.copy(f'{good_data_folder_path}'+filename , bad_data_folder_path)

In [27]:
validatecolumn(NumberofColumns)

In [28]:
def validate_missing_column():
    for filename in os.listdir(good_data_folder_path):
        csv = pd.read_csv(f'{good_data_folder_path}'+filename)
        count=0
        for column in csv:
            if (len(csv[column])- csv[column].count()) == len(csv[column]):
                count +=1
                shutil.copy(f'{good_data_folder_path}'+filename , bad_data_folder_path)
                break
        if count==0:
            csv.rename(columns = {'Unnamed: 0': "wafer"}, inplace = True)
            csv.to_csv(f'{good_data_folder_path}'+ filename, index = None , header=True)

In [29]:
validate_missing_column()

In [ ]:
#Next Steps
#Adding all data into Database
# Empty good_data folder from the computer
#Archive the Bad_data

In [30]:
# Merging or Combining all Data
li = []
for filename in os.listdir(good_data_folder_path):
    csv = pd.read_csv(f'{good_data_folder_path}'+filename)
    li.append(csv)

combined_data = pd.concat(li , axis = 0, ignore_index=True)
combined_data.to_csv('main_data.csv')